### Part 1 - Building the *CNN*

In [1]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# Initialising the CNN
classifier = Sequential()

#### Step 1 - Convolution
Key part is the size of the number of filters and the size (also the dimensions and the channels of the image). In the initial layers we don't need to have filters of large sizes since we have large info about the input data already, but as we go deep in the network, we need to preserve more and more features, hence we use large filter sizes, 64, 128 etc. ideally.
The number of or the type of filters specify the number of feature maps -> each type of filter is for some one kind of feature that we are trying to detect in an image, edge, curve whatever it would be, hence the number of feature maps -> after the filter is successfully convolved with the input image.

In [3]:
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

#### Step 2 - Pooling
To reduce the size of the feature maps, apply max-pooling to get a new feature map with a reduced size by only taking max vaule.
If we don't do this, we will get a high dimension feature vector as input to the fully connected network, that adds to computation.
Reduce the complexity of the model by two, without affecting/reducing the performance.

In [4]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [5]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#### Step 3 - Flattening
What if you directly apply flattening to the input image?
That way we have each node of the FC layer represent each pixel, and there is no information about the pixel adjacent to it, in other words, no spatial information is preserved. 
Since we want a node to detect a feature, we use convolution then pooling to get a feature map which can be flattened. 

In [6]:
classifier.add(Flatten())

#### Step 4 - Full Connection
The choice of the number nodes is a hyperparam, that can be tuned only by experiment, but should be ^2. Shouldn't be too less, nor too large.

In [7]:
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid')) #output layer 
#since binary we use sigmoid, if multi-class use softmax

In [8]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images

Image augmentation is important to avoid overfitting, so some form of pre-processing is necessary (use Keras for that). <br>
Computer Vision requires a large dataset to get good at detecting patterns, so with a limited dataset, augmentation is a trick which helps in creating many batches, applies geometrical transformations randomly like it rotates, flips, shearing etc. and diversifies the dataset. 

In [12]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Resources/cnn_dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('Resources/cnn_dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 1, #takes 30 mins for 1
                         validation_data = test_set,
                         validation_steps = 2000)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
8000/8000 [==============================] - 2029s 254ms/step - loss: 0.0585 - acc: 0.9794 - val_loss: 1.0198 - val_acc: 0.7966
